In [1]:
import os
import time

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import (
    Sequential,
    layers,
    optimizers,
    losses,
    regularizers,
    callbacks
)
import tensorflow_hub as hub

In [4]:
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
    raise SystemError('GPU not found!')
else:
    print(f'GPU found => {device_name}')

GPU found => /device:GPU:0


# Constants

In [7]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

TRAIN_URL = "gs://practical-ml-vision-book/flowers_tfr/train-*"
VALID_URL = "gs://practical-ml-vision-book/flowers_tfr/valid-*"

BATCH_SIZE = 512 if device_name != "/device:GPU:0" else 32
CLASS_NAMES = 'daisy dandelion roses sunflowers tulips'.split()

MODEL_URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"


# Helper Functions

In [6]:
def get_logdir():
    run_id = time.strftime('run_%Y_%m_%d-%H%M%S')
    logdir = os.path.join('..', '..', 'reports', 'logs', run_id)
    return logdir

# Preprocessor

In [ ]:
class _Preprocessor:
    def __init__(self):
        pass

    def read_from_tfr(self, proto):
        feature_description = {
            "image": tf.io.VarLenFeature(tf.float32),
            "shape": tf.io.VarLenFeature(tf.int64),
            "label": tf.io.FixedLenFeature([], tf.string, default_value=""),
            "label_int": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        }

        record = tf.io.parse_single_example(proto, feature_description)
        shape = tf.

# Transfer Learning with MobileNet